# Techniques for the Assignment 2/2

JD Sports and Frasers Group as used as exmaples in this session.

## Download the required data

You need to download the daily prices from 02/03/2020 to 28/02/2022. Then set the last 20 days as the test set and the first 485 days (for this example, yours may vary) as the training set.

In [ ]:
# import required libraries or commands within a library

In [ ]:
pip install yfinance

In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

In [ ]:

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
#from sklearn.metrics import mean_absolute_percentage_error as MAPE
import statsmodels.api as sa
import statsmodels.formula.api as sfa
from statsmodels.tsa.api import ExponentialSmoothing as TES 
from statsmodels.tsa.api import SimpleExpSmoothing as SES
from statsmodels.tsa.api import Holt as DES

In [ ]:
# Download JD Sports prices
Smurfit=yf.download('SKG.L',start='2020-03-02',end='2022-03-01')
Mondi=yf.download('MNDI.L',start='2020-03-02',end='2022-03-01')
FTSE=yf.download('^FTSE',start='2020-03-02',end='2022-03-01')

In [ ]:
Smurfit

In [ ]:
Mondi

In [ ]:
FTSE

In [ ]:
#show the index of DataFrame
Smurfit.index

In [ ]:
#check is there any point of Smurfit index different from Mondi index
Smurfit.index.difference(Mondi.index)

In [ ]:
#check is there any point of Smurffit index different from FTSE index
Smurfit.index.difference(FTSE.index)

In [ ]:
#What's the difference between this line and the above one?
FTSE.index.difference(Smurfit.index)

In [ ]:
#drop the row
Smurfit=Smurfit.drop(['2020-12-22'])
Mondi=Mondi.drop(['2020-12-22'])

In [ ]:
Smurfit.head(1)

### Training and Test sets

In [ ]:
# Drop the columns we don't need
Smurfit=Smurfit.drop(columns=['Open','High','Low','Close','Volume'])
Mondi=Mondi.drop(columns=['Open','High','Low','Close','Volume'])
FTSE=FTSE.drop(columns=['Open','High','Low','Close','Volume'])

In [ ]:
# Training sets
Smurfittr=Smurfit.iloc[0:484]
Monditr=Mondi.iloc[0:484]
FTSEtr=FTSE.iloc[0:484]

In [ ]:
# Test set
Smurfitte=Smurfit.iloc[484:504]
Mondite=Mondi.iloc[484:504]
FTSEte=FTSE.iloc[484:504]

### Daily Returns

In [ ]:
returns1=(Smurfittr['Adj Close']-Smurfittr.shift(1)['Adj Close'])/Smurfittr.shift(1)['Adj Close']
returns2=(Monditr['Adj Close']-Monditr.shift(1)['Adj Close'])/Monditr.shift(1)['Adj Close']
returns3=(FTSEtr['Adj Close']-FTSEtr.shift(1)['Adj Close'])/FTSEtr.shift(1)['Adj Close']

In [ ]:
Smurfittr['Return']=returns1
Monditr['Return']=returns2
FTSEtr['Return']=returns3

In [ ]:
Smurfitte['Return']=(Smurfitte['Adj Close']-Smurfitte.shift(1)['Adj Close'])/Smurfitte.shift(1)['Adj Close']
Smurfitte.iloc[0,1]=(Smurfitte.iloc[0,0]-Smurfittr.iloc[-1,0])/Smurfittr.iloc[-1,0]
Mondite['Return']=(Mondite['Adj Close']-Mondite.shift(1)['Adj Close'])/Mondite.shift(1)['Adj Close']
Mondite.iloc[0,1]=(Mondite.iloc[0,0]-Monditr.iloc[-1,0])/Monditr.iloc[-1,0]
FTSEte['Return']=(FTSEte['Adj Close']-FTSEte.shift(1)['Adj Close'])/FTSEte.shift(1)['Adj Close']
FTSEte.iloc[0,1]=(FTSEte.iloc[0,0]-FTSEtr.iloc[-1,0])/FTSEtr.iloc[-1,0]

## Descriptive Analytics

In [ ]:
Smurfittr['Adj Close'].plot(figsize=(15,10), label='Smurfit')
Monditr['Adj Close'].plot(label='Mondi')
FTSEtr['Adj Close'].plot(label='FTSE')
plt.legend()
plt.show()

In [ ]:
#define a modified FTSE
mFTSEtr=pd.DataFrame()
mFTSEtr['m Adj Close']=FTSEtr['Adj Close']/10

In [ ]:
Smurfittr['Adj Close'].plot(figsize=(15,10), label='Smurfit')
Monditr['Adj Close'].plot(label='Mondi')
mFTSEtr['m Adj Close'].plot(label='mFTSE')
plt.legend()
plt.show()

In [ ]:
Smurfittr['Return'].plot(figsize=(15,10), label='Smurfit r')
Monditr['Return'].plot(label='Mondi r')
FTSEtr['Return'].plot(label='FTSE r')
plt.legend()
plt.show()

In [ ]:
#Empirical Rules, Page 75 of Lecture 1
Smurfittr['Return'].plot(figsize=(15,10), label='Smurfit r')
plt.axhline(y=Smurfittr['Return'].mean()+2*Smurfittr['Return'].std(),label='+2s',color='g')
plt.axhline(y=Smurfittr['Return'].mean()-2*Smurfittr['Return'].std(),label='-2s',color='r')
plt.legend()
plt.show()

In [ ]:
#select rows by value 
pss=Smurfittr['Return'].mean()+2*Smurfittr['Return'].std()
Smurfittr.loc[Smurfittr['Return']>=pss]

In [ ]:
nss=Smurfittr['Return'].mean()-2*Smurfittr['Return'].std()
Smurfittr.loc[Smurfittr['Return']<=nss]

In [ ]:
Smurfittr.loc[Smurfittr['Return']==Smurfittr['Return'].min()]

### Statistics

In [ ]:
Smurfittr.describe()

In [ ]:
Monditr.describe()

In [ ]:
FTSEtr.describe()

What can you find by comparing the above tables?

In [ ]:
# Return to risk
Smurfittr['Return'].mean()/Monditr['Return'].std()

### Boxplot

In [ ]:
lgb=Smurfittr.boxplot(column='Return',whis=[0,100],grid=False)

lgb.set_title("Smurfit Daily Returns",fontsize=12)
lgb.set_xlabel('Smurfit', fontsize=10)
lgb.set_ylabel('Return', fontsize=10)
lgb.set_xticks([])
plt.show()

In [ ]:
#Creturns=pd.concat([Smurfittr['Return'],Monditr['Return'],FTSEtr['Return']],axis=1)
#Creturns.columns=['smurfit','Mondi','FTSE']

In [ ]:
AllTrainingReturns=pd.DataFrame({'Smurfit r':Smurfittr['Return'],'Mondi r':Monditr['Return'],'FTSE r':FTSEtr['Return']})

In [ ]:
AllTrainingReturns

In [ ]:
AllTrainingReturns.boxplot(whis=[0,100],grid=False)
plt.title('Comparing Daily Returns')
plt.ylabel('Return')
plt.show()

### Histogram

In [ ]:
Smurfittr.hist(column='Return',bins=20,grid=False)

plt.title("Smurfit Daily Returns",fontsize=12)
plt.xlabel('Bin', fontsize=10)
plt.ylabel('Frequency', fontsize=10)

plt.axvline(Smurfittr['Return'].mean())
plt.text(Smurfittr['Return'].mean()*1.1,150,'Mean: {:,.4f}'.format(Smurfittr['Return'].mean()))

plt.axvline(Smurfittr['Return'].median(),color='r')
plt.text(Smurfittr['Return'].median()*1.2,110,'Median: {:,.4f}'.format(Smurfittr['Return'].median()))

plt.show()

In [ ]:
# it's better to draw histograms individually by the above codes

AllTrainingReturns.hist(bins=40,grid=False)
plt.show()

## Regression

In [ ]:
# The three dataframes have the same column names, we should change the names and merge them together
Smurfittr.columns=['Smurfit_Price','Smurfit_Return']
Monditr.columns=['Mondi_Price','Mondi_Return']
FTSEtr.columns=['FTSE_Price','FTSE_Return']
#concatenate them by columns
OverallTraining=pd.concat([Smurfittr,Monditr,FTSEtr],axis=1)

In [ ]:
OverallTraining.head()

In [ ]:
OverallTraining.plot.scatter(x='Smurfit_Price',y='Mondi_Price',alpha=0.7)
plt.show()

In [ ]:
pd.plotting.scatter_matrix(OverallTraining.iloc[:,[0,2,4]], alpha=0.7)
plt.show()

In [ ]:
OverallTraining.iloc[:,[0,2,4]].corr(method='pearson')

In [ ]:
pd.plotting.scatter_matrix(OverallTraining.iloc[:,[1,3,5]], alpha=0.7)
plt.show()

In [ ]:
OverallTraining.iloc[:,[1,3,5]].corr(method='pearson')

In [ ]:
Smurfit_Mondi_reg=sfa.ols('Smurfit_Price ~ Mondi_Price', data=OverallTraining).fit()

In [ ]:
Smurfit_Mondi_reg.summary()

In [ ]:
sa.graphics.plot_fit(Smurfit_Mondi_reg,exog_idx='Mondi_Price',vlines=False)
plt.ylabel("Smurfit Price")
plt.xlabel("Mondi Price")
plt.title("Smurfit vs Mondi")
plt.show()

In [ ]:
Smurfit_FTSE_rreg=sfa.ols('Smurfit_Return ~ FTSE_Return', data=OverallTraining).fit()

In [ ]:
Smurfit_FTSE_rreg.summary()

In [ ]:
sa.graphics.plot_fit(Smurfit_FTSE_rreg,exog_idx='FTSE_Return',vlines=False)
plt.ylabel("Smurfit Return")
plt.xlabel("FTSE Return")
plt.title("Smurfit vs FTSE")
plt.show()

In [ ]:
# Multiple Regression
Smurfit_Mondi_FTSE_reg=sfa.ols('Smurfit_Price ~ Mondi_Price + FTSE_Price', data=OverallTraining).fit()

In [ ]:
Smurfit_Mondi_FTSE_reg.summary()

## Time Series

In [ ]:
Smurfitte.columns=['Smurfit_Price','Smurfit_Return']
Mondite.columns=['Mondi_Price','Mondi_Return']
FTSEte.columns=['FTSE_Price','FTSE_Return']

### Moving Average

In [ ]:
ma5=Smurfittr.rolling(5).mean()

In [ ]:
ma20=Smurfittr.rolling(20).mean()

In [ ]:
Smurfittr['Smurfit_Price'].plot(figsize=(15,10), title= 'Moving average', label='Price')
ma5['Smurfit_Price'].plot(label='MA5')
ma20['Smurfit_Price'].plot(label='MA20')
plt.legend()
plt.ylabel('Price',fontsize=15)
plt.show()

In [ ]:
#creat forecasts
Smurfitma5fcsts=pd.Series([ma5['Smurfit_Price'][-1]]*20)
Smurfitma5fcsts.index=Smurfitte.index

In [ ]:
Smurfittr['Smurfit_Price'].plot(figsize=(15,10),label='train',title='5-day Moving Average')
Smurfitte['Smurfit_Price'].plot(label='test')
ma5['Smurfit_Price'].plot(label='fitted')
Smurfitma5fcsts.plot(label='forecasts')
plt.legend()
plt.show()

In [ ]:
MAE(Smurfitte['Smurfit_Price'],Smurfitma5fcsts)

In [ ]:
MSE(Smurfitte['Smurfit_Price'],Smurfitma5fcsts)

In [ ]:
print(
    'MAE: {:,.4f}'.format(MAE(Smurfitte['Smurfit_Price'],Smurfitma5fcsts)),
    '\nMSE: {:,.4f}'.format(MSE(Smurfitte['Smurfit_Price'],Smurfitma5fcsts))
                         )

### Single Exponential Smoothing

In [ ]:
Smurfitses=SES(Smurfittr['Smurfit_Price'],initialization_method="estimated").fit()

In [ ]:
Smurfitsesfcsts=Smurfitses.forecast(20)
Smurfitsesfcsts.index=Smurfitte.index

In [ ]:
Smurfittr['Smurfit_Price'].plot(figsize=(15,10),marker='o',color='black',label='train',title='Single Exponential Smoothing')
Smurfitte['Smurfit_Price'].plot(marker='o',color='brown',label='test')
Smurfitses.fittedvalues.plot(marker='v', color='blue',label=r'$\alpha=%s$'%Smurfitses.model.params['smoothing_level'])
Smurfitsesfcsts.plot(marker='v', color='green',label='forecasts')
plt.legend()
plt.show()

In [ ]:
print(
    'MAE: {:,.4f}'.format(MAE(Smurfitte['Smurfit_Price'],Smurfitsesfcsts)),
    '\nMSE: {:,.4f}'.format(MSE(Smurfitte['Smurfit_Price'],Smurfitsesfcsts))
)

### Double Exponential Smoothing

In [ ]:
Smurfitdes=DES(Smurfittr['Smurfit_Price'],initialization_method="estimated").fit()

In [ ]:
Smurfitdesfcsts=Smurfitdes.forecast(20)
Smurfitdesfcsts.index=Smurfitte.index

In [ ]:
Smurfittr['Smurfit_Price'].plot(figsize=(15,10),marker='o',color='black',label='train',title='Double Exponential Smoothing')
Smurfitte['Smurfit_Price'].plot(marker='o',color='brown',label='test')
Smurfitdes.fittedvalues.plot(marker='v', color='blue',label='fitted')
Smurfitdesfcsts.plot(marker='v', color='green',label='forecasts')
plt.legend()
plt.show()

In [ ]:
print(
    'MAE: {:,.4f}'.format(MAE(Smurfitte['Smurfit_Price'],Smurfitdesfcsts)),
    '\nMSE: {:,.4f}'.format(MSE(Smurfitte['Smurfit_Price'],Smurfitdesfcsts))
)

### Holt's Winters (Additive)

In [ ]:
Smurfittesa=TES(Smurfittr['Smurfit_Price'],seasonal_periods=4, trend='add', seasonal='add', use_boxcox=True, initialization_method="estimated").fit()
Smurfittesfcstsa=Smurfittesa.forecast(20)
Smurfittesfcstsa.index=Smurfitte.index

In [ ]:
Smurfittr['Smurfit_Price'].plot(figsize=(15,10),marker='o',color='black',label='train',title="Holt's Winters Method (Additive)")
Smurfitte['Smurfit_Price'].plot(marker='o',color='brown',label='test')
Smurfittesa.fittedvalues.plot(marker='v', color='blue',label='fitted')
Smurfittesfcstsa.plot(marker='v', color='green',label='forecasts')
plt.legend()
plt.show()

In [ ]:
print(
    'MAE: {:,.4f}'.format(MAE(Smurfitte['Smurfit_Price'],Smurfittesfcstsa)),
    '\nMSE: {:,.4f}'.format(MSE(Smurfitte['Smurfit_Price'],Smurfittesfcstsa))
)

### Holt's Winters (Multiplicative)

In [ ]:
Smurfittesm=TES(Smurfittr['Smurfit_Price'],seasonal_periods=4, trend='add', seasonal='mul', use_boxcox=True, initialization_method="estimated").fit()
Smurfittesfcstsm=Smurfittesm.forecast(20)
Smurfittesfcstsm.index=Smurfitte.index

In [ ]:
Smurfittr['Smurfit_Price'].plot(figsize=(15,10),marker='o',color='black',label='train',title="Holt's Winters Method (Multiplicative)")
Smurfitte['Smurfit_Price'].plot(marker='o',color='brown',label='test')
Smurfittesm.fittedvalues.plot(marker='v', color='blue',label='fitted')
Smurfittesfcstsm.plot(marker='v', color='green',label='forecasts')
plt.legend()
plt.show()

In [ ]:
print(
    'MAE: {:,.4f}'.format(MAE(Smurfitte['Smurfit_Price'],Smurfittesfcstsm)),
    '\nMSE: {:,.4f}'.format(MSE(Smurfitte['Smurfit_Price'],Smurfittesfcstsm))
)